In this Notebook I will add the Spotify track URIs to the billboards data set.

I will use the spotipy package because the querys are less sensitive and easier to create than connecting directly to the Spotify API.

Methodology:

Import packages & .env to store API keys Easily done thanks to the lessons learnt from the API mini project. DONE
Authenticate myself via the client credentials through the spotipy package After reading through the documentation this methd of connecting to the API seems easier and fewere errors with the query portion of the GET request.
Create a funtion to get URIs for songs from the JSON output of the API This is proving to be very dificult. There are many songs that are not
Apply the funtion to the Billboards dataset
Check for missing values, duplicates, etc. and make changes to the function as required

In [80]:
# get api key from your .env file

import os

from dotenv import load_dotenv

from pathlib import Path

 

dotenv_path = Path('./SpotifyAPIKey.env')

load_dotenv(dotenv_path=dotenv_path)

 

#load_dotenv()

API_CLIENT_ID = os.getenv('SPOTIFY_CLIENT_ID')
API_CLIENT_SECRET = os.getenv('SPOTIFY_CLIENT_SECRET')
 
print(API_CLIENT_ID)
print(API_CLIENT_SECRET)

3f7295d24d8041688f494196baeb607f
42896ec37b22446680c5a485858a4843


In [81]:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

#https://spotipy.readthedocs.io/en/2.19.0/

In [82]:
client_credentials_manager = SpotifyClientCredentials(client_id=API_CLIENT_ID,client_secret=API_CLIENT_SECRET)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

def URI(titleName, artistName):
    track = sp.search(q="artist:" + f"{artistName}"+" track:"+f"{titleName}",type='track',limit=50)
    result = track['tracks']['items']
    result = sorted(result, key = lambda i: i['popularity'], reverse = True)
    
    for items in result:
        for artist in items['artists']:
            if artist['name'].lower() == artistName.lower():
                uri = items['id']
                return(uri)
            elif artist['name'].lower() in artistName.lower():
                uri = items['id']
                return uri
            
    track = sp.search(titleName,type='track',limit=50)
    result = track['tracks']['items']
    result = sorted(result, key = lambda i: i['popularity'], reverse = True)
    
    for items in result:
        for artist in items['artists']:
            if artist['name'].lower() == artistName.lower():
                uri = items['id']
                return uri

    for items in result:
        for artist in items['artists']:
            if artist['name'].lower() in artistName.lower():
                uri = items['id']
                return uri
            
    track = sp.search(titleName,type='track',limit=50)
    result = track['tracks']['items']
    result = sorted(result, key = lambda i: i['popularity'], reverse = True)
    
    for items in result:
        if items['name'].lower() == titleName.lower():
            uri = items['id']
            return uri
        elif items['name'].lower() in titleName.lower():
            uri = items['id']
            return uri
    

print(URI('caroline', "amine"))

5hTpBe8h35rJ67eAWHQsJx


In [89]:
print(URI("the way you love me", "faith hill"))

6vgN6KGstss0GnBbKlMNK9


In [90]:
df=pd.read_csv('BillboardFullDataset.csv')

In [91]:
#As discovered there are some issues with some of the data. They contain \xa0 in place of spaces. I change those here. 

df['Title'] = df['Title'].apply(lambda x: str(x).replace(u'\xa0', u' '))
df['Artist'] = df['Artist'].apply(lambda x: str(x).replace(u'\xa0', u' '))

In [92]:
df.head(10)

,Unnamed: 0,Artist,Title
0,0,Lifehouse,Hanging By A Moment
1,1,Alicia Keys,Fallin’
2,2,Janet Jackson,All For You
3,3,Train,Drops Of Jupiter (Tell Me)
4,4,Jennifer Lopez feat. Ja Rule,I’m Real
5,5,matchbox twenty,If You’re Gone
6,6,Eve feat. Gwen Stefani,Let Me Blow Ya Mind
7,7,Dido,Thank You
8,8,Lenny Kravitz,Again
9,9,Destiny’s Child,Independent Women


In [93]:
df = df[['Title','Artist']]

df['Title'] = df['Title'].str.lower()
df['Artist'] = df['Artist'].str.lower()

In [94]:
df.head()

,Title,Artist
0,hanging by a moment,lifehouse
1,fallin’,alicia keys
2,all for you,janet jackson
3,drops of jupiter (tell me),train
4,i’m real,jennifer lopez feat. ja rule


In [96]:
def append_URIs(dataframe):
    for lab, row in dataframe.iterrows():
        dataframe.loc[lab, 'URI'] = URI(row['Title'],row['Artist'])

In [97]:
append_URIs(df)

In [98]:
df.describe()

,Title,Artist,URI
count,1875,1875,1873
unique,1746,1188,1802
top,beautiful,rihanna,0l3GEbH3CviUytD6iX4wzg
freq,4,17,3


There are issues here. There are missing URIs and duplicate URIs.

I'll first attack the missing URIs - I think these can be solved by changing the functions slightly to make sure there are no name issues e.g. sounfs featuring multiple artist on billboard but only showing the main artist on spotify

Round 1: Stared with 151 missing URIs.

Round 2: After fixing 1 Artist name I am down to 150 missing URIs.

Round 3: I changed my append_URIs function to take in all lowercase values for artist. I am now down to missing 138 URIs

Round 4: I added some conditions to my function to also look through track names and match the artist and from there. I also added a condition to look at lowercase songs. I am down to 14 missing URIs.

Note: this is weird.. function to get the uri works on some of these missing values so there must be an issue with the append uri's function.

In [99]:
missing = df.loc[df['URI'].isna()]

In [100]:
missing

,Title,Artist,URI
70,ain’t nothing `bout you,brooks and dunn,None
576,for you i will (condience),teddy geiger,None


In [101]:
#Dropping the last two NaN values.

df.dropna()

,Title,Artist,URI
0,hanging by a moment,lifehouse,0wqOReZDnrefefEsrIGeR4
1,fallin’,alicia keys,0KQx6HOpJueiSkztcS0r7D
2,all for you,janet jackson,5X8kkUaUlAyAUr9TYqDFTH
3,drops of jupiter (tell me),train,2hKdd3qO7cWr2Jo0Bcs0MA
4,i’m real,jennifer lopez feat. ja rule,6MLsahMK3VvH9NUwXOktdS
...,...,...,...
1870,more than my hometown,morgan wallen,5OELUCYgOHKFAvCERnAvfS
1871,lovin’ on you,luke combs,0nYvjcSlCgjcwogQAwIwNp
1872,said sum,moneybagg yo,3sKz6Sd72K0ofPWcJPPk6H
1873,slide,h.e.r. featuring yg,6MO2bfLHKykUgCChFdw91H


In [102]:
df.to_csv('BillboardWithURI.csv')